In [1]:
import pandas as pd
import numpy as np
import pickle
import random
from sklearn.preprocessing import LabelEncoder

In [ ]:
data[]

In [ ]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)


In [3]:
data = data.fillna(0)
data = data.replace('None', 0)

In [4]:
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [5]:
# 1 - 2%
# 2 - 3
# 3 - 31
# 4 - 24
# 5 - 38


num_test = {1: 0.02, 2:0.03, 3:0.31, 4: 0.24, 5:0.38}
test = []
for segment, num in num_test.items():
    test.append(data.loc[data['Segment'] == segment].sample(n=int(num*11213629)))
test = pd.concat(test)

In [ ]:
test_ids = list(pd.read_csv('my_test.csv')['id'])

In [ ]:
# data['gamecategory'].fillna('Games',inplace=True)
# data['subgamecategory'].replace('Puzzle', name,inplace=True)

# data['oblast'].fillna('Москва',inplace=True)
# data['city'].replace('Москва', name,inplace=True)

# data['os'].fillna('android',inplace=True)
# data['osv'].replace('10.0', name,inplace=True)
# data['osv_numerical'].replace('0.890756', name,inplace=True)

In [ ]:
# data[['bundle', 'bundle_hash']].value_counts(sort=True)

In [ ]:
# fillna = {}
# for i in t.columns:
#     a = t[i].value_counts(sort=True)
#     b = pd.DataFrame(a)
#     name = b.iloc[0].name
#     fillna[i] = name
#     t[i].fillna(name,inplace=True)
#     t[i].replace('None', name,inplace=True)

In [ ]:
# fillna = {}
# for i in data.columns:
#     a = data[i].value_counts(sort=True)
#     b = pd.DataFrame(a)
#     name = b.iloc[0].name
#     fillna[i] = name
#     data[i].fillna(name,inplace=True)
#     data[i].replace('None', name,inplace=True)

In [ ]:
# dict_test_cats = {}
# for i in set(t['gamecategory']):
#     cur_cat = t.loc[t['gamecategory'] == i]
#     dict_test_cats[i] = {}
#     for j in set(cur_cat['subgamecategory']):
#         num = cur_cat.loc[cur_cat['subgamecategory'] == j].shape[0]
#         dict_test_cats[i][j] = num


In [ ]:
data = data.dropna()

In [ ]:
# test_subs = []
# for i in dict_test_cats.keys():
#     sub_data = data.loc[data['gamecategory'] == i]
#     for j in dict_test_cats[i].keys():
#         num = dict_test_cats[i][j]
#         if  num > 0:
#             sub_sub_data = sub_data.loc[sub_data['subgamecategory'] == j]
#             if sub_sub_data.shape[0] <= num:
#                 print(num, sub_sub_data.shape[0], i, j)
#                 num =sub_sub_data.shape[0] 
#             sub_test = sub_sub_data.sample(n = num)
#             test_subs.append(sub_test)

In [7]:
test_ids = list(test['id'])
train = data.loc[~data['id'].isin(test_ids)]
print(train.shape, test.shape)

(33865163, 20) (10989353, 20)


<h1> Selecting & encoding features </h1>

In [ ]:
def encode_features(data):
    data['interp_game'] = data['interp_game'].astype(str)
    data['interp_subgame'] = data['interp_subgame'].astype(str)
    data['city'] = data['interp_subgame'].astype(str)
    data['oblast'] = data['oblast'].astype(str)
    data['os'] = data['os'].astype(str)
    data['shift'] = data['shift'].astype(str)

    data_dict = {}
    features_to_encoding = ['interp_game','oblast','interp_subgame','city','os','shift','day_of_week']
    using_features = ['Segment', 'shift', 'oblast', 'city', 'os', 'interp_game',
           'interp_subgame', 'osv_numerical',  'day_of_week',
           'is_weekend', 'hour', 'dist_from_msk_in_tz_hours','id']

    for feature in features_to_encoding:
        le = LabelEncoder()
        print(feature)
        data_dict[feature] = np.array(le.fit_transform(np.array(data[feature])))


    for category in set(using_features) - set(features_to_encoding):
        data_dict[category] = np.array(data[category])
    return data_dict
data_dict = encode_features(data)

In [ ]:
indexes_test = np.where(np.isin(data_dict['id'], test_ids))[0]
indexes_train = list(set(list(range(0, data_dict['id'].shape[0]))) - set(indexes_test))
train_data_dict = {}
test_data_dict = {}
for category in set(using_features):
    train_data_dict[category] = np.array(data_dict[category][indexes_train])
    train_data_dict[category] =  train_data_dict[category][::2]
    test_data_dict[category] = np.array(data_dict[category][indexes_test])
    


In [ ]:
data_dict.shape[0]

<h1> Balance dataset </h1>

In [ ]:
segments = list(range(1, 6))
size_segments = {}
for i in segments:
    size_segments[i] = np.where(train_data_dict['Segment'] == i)[0].shape[0]

max_size = np.max(list(size_segments.values()))

all_balanced_data = {}
for segment in segments:
    diff = size_segments[segment] - max_size
    
    if diff < 0 and abs(diff) > size_segments[segment]:
        cur_data_segment = {}
        indexes = np.where(train_data_dict['Segment'] == segment)[0] 
        for feature in using_features:
            cur_data_segment[feature] = train_data_dict[feature][indexes]
        final_data = {}
        count_repeated = abs(diff) // size_segments[segment]
        if count_repeated == 1:
            count_repeated += 1
        for feature in using_features:
            if feature not in all_balanced_data.keys():
                all_balanced_data[feature] = np.tile(cur_data_segment[feature], count_repeated)
            else:
                all_balanced_data[feature] = np.concatenate((all_balanced_data[feature], np.tile(cur_data_segment[feature], count_repeated)))
    elif diff < 0:
        cur_data_segment = {}
        indexes = np.where(train_data_dict['Segment'] == segment)[0] 
        for feature in using_features:
            cur_data_segment[feature] = train_data_dict[feature][indexes]
            
        indexes = np.array(random.sample(list(range(0, cur_data_segment['Segment'].shape[0])), abs(diff))).astype(int)
        for feature in using_features:
            if feature not in all_balanced_data.keys():
                all_balanced_data[feature] = np.concatenate((cur_data_segment[feature],cur_data_segment[feature][indexes]))
            else:
                tmp = np.concatenate((cur_data_segment[feature],cur_data_segment[feature][indexes]))
                all_balanced_data[feature] = np.concatenate((all_balanced_data[feature], tmp))
    else:
        cur_data_segment = {}
        indexes = np.where(train_data_dict['Segment'] == segment)[0] 
        for feature in using_features:
            cur_data_segment[feature] = train_data_dict[feature][indexes]
        for feature in using_features:
            if feature not in all_balanced_data.keys():
                all_balanced_data[feature] = cur_data_segment[feature]
            else:
                all_balanced_data[feature] = np.concatenate((all_balanced_data[feature], cur_data_segment[feature]))
    now_size = np.where(all_balanced_data['Segment'] == segment)[0].shape[0]
    print(f'Segment {segment} balanced. Was {size_segments[segment]} size. Now: {now_size}')

Segment 1 balanced. Was 697359 size. Now: 11855103
Segment 2 balanced. Was 1079837 size. Now: 10798370
Segment 3 balanced. Was 10710830 size. Now: 12926327
Segment 4 balanced. Was 8450810 size. Now: 12926327


In [ ]:
for i in using_features:
    all_balanced_data[i] = all_balanced_data[i][::5]

In [8]:
def fun(data_dict_cur):
    data_dict_cur['hour'] = data_dict_cur['hour'].astype(int)
    data_dict_cur['hour'] = (data_dict_cur['hour'] - np.min(data_dict_cur['hour'])) / np.max(data_dict_cur['hour']) - np.min(data_dict_cur['hour'])
    data_dict_cur['dist_from_msk_in_tz_hours'] = (data_dict_cur['dist_from_msk_in_tz_hours'] - \
                                                      np.min(data_dict_cur['dist_from_msk_in_tz_hours'])) / \
    np.max(data_dict_cur['dist_from_msk_in_tz_hours']) - np.min(data_dict_cur['dist_from_msk_in_tz_hours'])
    return data_dict_cur

In [9]:
train_data_dict = fun(train_data_dict)
test_data_dict = fun(test_data_dict)

In [17]:
train_data_dict

{'os': array([1, 2, 1, ..., 2, 2, 2]),
 'day_of_week': array([2, 5, 3, ..., 3, 6, 0]),
 'is_weekend': array([0, 1, 0, ..., 0, 1, 0]),
 'dist_from_msk_in_tz_hours': array([0.        , 0.22222222, 0.        , ..., 0.33333333, 0.        ,
        0.        ]),
 'interp_game': array([0, 8, 1, ..., 8, 8, 8]),
 'city': array([ 0, 13, 34, ..., 37, 37, 37]),
 'shift': array([1, 3, 1, ..., 4, 1, 1]),
 'osv_numerical': array([0.8907563 , 0.92156863, 0.96638655, ..., 0.90849673, 0.87581699,
        0.97385621]),
 'hour': array([0.60869565, 0.65217391, 0.13043478, ..., 0.95652174, 0.30434783,
        0.95652174]),
 'interp_subgame': array([ 0, 13, 34, ..., 37, 37, 37]),
 'oblast': array([48, 69, 90, ..., 54, 48, 65]),
 'id': array([       0,        2,        4, ..., 44854512, 44854514, 44854515]),
 'Segment': array([5, 5, 5, ..., 3, 3, 5])}

<h1> Train-validation-test dataset </h1>

In [18]:
def get_train_x_y(cur_data):
    x = np.empty((cur_data['Segment'].shape[0],len(using_features)-1))
    for i, feature in zip(range(0, 12),['shift', 'oblast', 'city', 'os', 'interp_game',
       'interp_subgame', 'osv_numerical',  'day_of_week',
       'is_weekend', 'hour', 'dist_from_msk_in_tz_hours','id']):
        x[:,i] = cur_data[feature]

    y = np.array(pd.get_dummies(cur_data['Segment']))
    return x, y

In [19]:
x_train, y_train = get_train_x_y(train_data_dict)
x_test, y_test = get_train_x_y(test_data_dict)

shift
oblast
city
os
interp_game
interp_subgame
osv_numerical
day_of_week
is_weekend
hour
dist_from_msk_in_tz_hours
id
shift
oblast
city
os
interp_game
interp_subgame
osv_numerical
day_of_week
is_weekend
hour
dist_from_msk_in_tz_hours
id


In [12]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33, random_state=44)




In [ ]:
y_train

In [ ]:
X_train

In [ ]:
X_train

In [13]:
from catboost import CatBoostClassifier, Pool
test_data = catboost_pool = Pool(X_train, 
                                 y_train)
cat_features = [0]
train_dataset = Pool(data=X_train,
                     label=y_train)

eval_dataset = Pool(data=X_valid,
                    label=y_valid)

test_dataset = Pool(data=x_test,
                    label=y_test)
model = CatBoostClassifier(iterations=10,
                           learning_rate=0.5,
                           depth=10,
                           loss_function='MultiCrossEntropy',
                          l2_leaf_reg = 0.05,
                           
                          )

# Fit model
model.fit(train_dataset, eval_set=eval_dataset,verbose_eval=True,use_best_model=True,
         early_stopping_rounds=5)

model2 = CatBoostClassifier(iterations=10,
                           learning_rate=0.15,
                           depth=10,
                           loss_function='MultiCrossEntropy',
                          l2_leaf_reg = 0.02,
                          )
model2.fit(train_dataset, eval_set=eval_dataset,verbose_eval=True,use_best_model=True,
         early_stopping_rounds=5, init_model=model)
# Get predicted classes
#preds_class = model.predict(test_dataset)
# Get predicted probabilities for each class
preds_proba = model2.predict_proba(x_test)

0:	learn: 0.5006353	test: 0.5006297	best: 0.5006297 (0)	total: 4.12s	remaining: 37.1s
1:	learn: 0.4309862	test: 0.4309580	best: 0.4309580 (1)	total: 8.3s	remaining: 33.2s
2:	learn: 0.4005583	test: 0.4005148	best: 0.4005148 (2)	total: 12.4s	remaining: 28.9s
3:	learn: 0.3830548	test: 0.3829906	best: 0.3829906 (3)	total: 16.4s	remaining: 24.6s
4:	learn: 0.3744401	test: 0.3743616	best: 0.3743616 (4)	total: 20.7s	remaining: 20.7s
5:	learn: 0.3699885	test: 0.3698727	best: 0.3698727 (5)	total: 24.6s	remaining: 16.4s
6:	learn: 0.3670408	test: 0.3669159	best: 0.3669159 (6)	total: 28.7s	remaining: 12.3s
7:	learn: 0.3647797	test: 0.3646420	best: 0.3646420 (7)	total: 32.8s	remaining: 8.2s
8:	learn: 0.3639202	test: 0.3637765	best: 0.3637765 (8)	total: 36.9s	remaining: 4.11s
9:	learn: 0.3630247	test: 0.3628694	best: 0.3628694 (9)	total: 41s	remaining: 0us

bestTest = 0.3628693628
bestIteration = 9

0:	learn: 0.3626235	test: 0.3624668	best: 0.3624668 (0)	total: 3.99s	remaining: 35.9s
1:	learn: 0.3623

In [ ]:
#array([0.56572721, 0.60320205, 0.58465289, 0.6038919 , 0.54682868])

In [14]:

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds_proba,  multi_class='ovr', average=None)

array([0.69857637, 0.84071803, 0.72615407, 0.73532001, 0.78249558])

In [ ]:
y_test.shape

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
 
cat = CatBoostClassifier(iterations=7,
                           learning_rate=0.7,
                           depth=10,
                           loss_function='MultiCrossEntropy',
                          l2_leaf_reg = 0.05,
                          )
ovr = OneVsRestClassifier(estimator=cat)
ovr.fit(X_train, y_train)
 
preds_proba = ovr.predict_proba(test_x)
 

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, preds_proba,  multi_class='ovr', average=None)

In [ ]:
#array([0.56572721, 0.60320205, 0.58465289, 0.6038919 , 0.54682868])

In [ ]:
X_test = np.array(X_test)
predicts = np.array(ovr.predict(X_test))
y_test_int = np.argmax(np.array(y_test), axis=1)+1
y_prob_int = np.argmax(np.array(predicts), axis=1)+1

In [ ]:
predicts

In [ ]:
with open('test_df_interp_bundle_osv_tz.pickle', 'rb') as f:
    test_no_target = pickle.load(f)


In [ ]:
test_no_target = test_no_target.fillna(0)
test_no_target = test_no_target.replace('None', 0)

In [ ]:
test_no_target_dict = encode_features(test_no_target)
test_no_target_dict